In [1]:
def find_in_jasoseol(search_month = 1, *search_by_name):

    import requests
    from bs4 import BeautifulSoup as BS
    import json
    import datetime
    import copy
    import pandas as pd
    import numpy as np

     # 함수 입력 값 컨트롤하기 (search_month, search_by_name)
    search_year = datetime.date.today().year
    search_day = datetime.date.today().day

    # 검색 '월' error handling
    if type(search_month) != int:
        print("잘못된 값입니다.")
        return
    elif (search_month > 12) :
        print("12개월 이내의 공고만 검색가능합니다.")
        return
    elif datetime.date.today().month - search_month < 1 :
        search_month = datetime.date.today().month - search_month + 12
        search_year-= 1 
    else:
        search_month = datetime.date.today().month - search_month

    # 검색 조건 결정
    start_date = datetime.date(search_year, search_month, search_day)
    end_date = datetime.date.today()

    # 기업명 결정하기
    if search_by_name == ():
        search_by_name = ""
    else: search_by_name = search_by_name[0]

    # jasoseol 사이트에서 채용 달력 정보 받아오기
    data = {
        'end_time':"{}T15:00:00.000Z".format(end_date),
        'start_time':"{}T15:00:00.000Z".format(start_date)
    }

    resp_jasoseol_calendar = requests.post("https://jasoseol.com/employment/calendar_list.json", data=data)
    calendar_all = pd.DataFrame(resp_jasoseol_calendar.json()["employment"])

    # 기업명이 입력되지 않았을 때 > 채용 달력 정보만 제공하고 return
    if search_by_name == '':
        return calendar_all

    # 기업명이 입력되었을 때 > 
    #  1) Detailed 공고 정보 제공
    #    가. 찾는 기업의 'id' 찾기
    i=0
    id_list = []
    flag=-1

    for i in range(len(resp_jasoseol_calendar.json()["employment"])):
        if search_by_name == resp_jasoseol_calendar.json()["employment"][i]["name"]:
            id_list.append(resp_jasoseol_calendar.json()["employment"][i]["id"])
            flag+=1

    # error handling (공고가 없을 경우!)
    if flag == -1 :
        print("해당 기간 중 찾으시는 기업의 채용 공고가 없습니다.")
        return calendar_all

    #    나. (jasoseol_detail) 기업 id를 통해 구체적 채용 정보 가져오기
    detail_each = []
    group_id = 0
    for i in range(len(id_list)):
        data = {"employment_company_id":id_list[i]}
        resp_jasoseol_detail = requests.post("https://jasoseol.com/employment/get.json", data=data)
        detail_each.append(resp_jasoseol_detail.json())
        group_id = resp_jasoseol_detail.json()['recruit_refs'][0]['company_group_id']

    detail_all = pd.DataFrame(detail_each)


    #  2) (data_lab) 기업 공고 지원자 정보 가져오기
    #    가. log-in session 만들기
    applicant_each = []
    applicant_all = []
    login_session = requests.Session()
    data = {
        'email': 'gdpark92@gmail.com',
        'password':'qkfka123!@'        
    }
    resp_data_lab = login_session.post("https://jasoseol.com/sign/log_in.json", data=data)
    
    #    나. 기업 공고 지원자 정보 가져오기
    for i in range(len(id_list)):
        headers = {
            'Origin': 'https://jasoseol.com',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'Referer':"https://jasoseol.com/datalab/{}?ec={}".format(group_id, id_list[i]),
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
        }
        data = {"company_group_id":str(group_id)}
        resp_data_lab = login_session.post('https://jasoseol.com/datalab/get_data.json', data=data, headers=headers)
        applicant_all.append(resp_data_lab.json()["result"]["employment_companies"][0]["employments"])
  
    # 최종 결과를 받는 List return
    result=[]
    result.append(calendar_all)
    result.append(detail_all)
    result.append(applicant_all)

    return result




In [2]:
def find_salary(search_by_name):
    
    search_by_name=search_by_name.strip()

    # import
    import requests
    from bs4 import BeautifulSoup as BS
    import json

    # 결과값을 담을 변수 선언
    result_li_alio = []
    result_li_fss = []

    salary_data ={}
    j=0
    k=0
    flag=0

    while(1):
        url_alio = "https://kreditjob.com/api/company/alioList?alioType=&order=ASC&pagingIndex={}&pagingNum=20&sortAlio=CMPN_NM".format(j)
        resp_alio = requests.get(url_alio)

        # 자료 값을 다 찾았을 때 loop 탈출
        if len(resp_alio.json()["data"]) == 0:
            break

        # 20개 단위로 resul_li 에 추가하기
        result_li_alio.append((resp_alio.json()["data"]))   

        # 기업명으로 검색하기
        for i in range(len(result_li_alio[k])):        
            if result_li_alio[k][i]["CMPN_NM"] == search_by_name:
                salary_data = {
                    "id":result_li_alio[k][i]["CMPN_NM"], 
                    "rookey_salary":result_li_alio[k][i]["ROOKEY_SALARY_YY"],  
                    "avg_salary":result_li_alio[k][i]["AVG_SALARY_YY"]
                }
                flag = 1
                break    

        # 다음 반복을 위한 변수값 재설정        
        j+=20
        k+=1

        # 검색에 성공했을 때 loop 탈출
        if flag == 1:
            break

    # 사기업 검색을 위한 변수 _init_
    j=0
    k=0
    flag=0     

    # 공기업에서 찾은 경우 검색 생략
    if flag == 0:
        while(1):
            url_fss = "https://kreditjob.com/api/company/fssList?fssType=&order=ASC&pagingIndex={}&pagingNum=20&sortFss=CMPN_NM".format(j)
            resp_fss = requests.get(url_fss)

            # 자료 값을 다 찾았을 때 loop 탈출
            if len(resp_fss.json()["data"]) == 0:
                break

            # 20개 단위로 resul_li 에 추가하기
            result_li_fss.append((resp_fss.json()["data"]))   

            # 기업명으로 검색하기
            for i in range(len(result_li_fss[k])):        
                if result_li_fss[k][i]["CMPN_NM"] == search_by_name:
                    salary_data = {
                        "id":result_li_fss[k][i]["CMPN_NM"], 
                        "rookey_salary":result_li_fss[k][i]["ROOKEY_SALARY_FSS"],  
                        "avg_salary":result_li_fss[k][i]["AVG_SALARY_FSS"]
                    }
                    flag = 1
                    break    

            # 다음 반복을 위한 변수값 재설정        
            j+=20
            k+=1

            # 검색에 성공했을 때 loop 탈출
            if flag == 1:
                break

    # 검색에 성공했을 때 값 return
    if flag == 1:
        print("기업명 :  {}".format(search_by_name), end="\n")
        print(" - 신입 연봉 : {} 원".format(int(salary_data["rookey_salary"])))
        print(" - 평균 연봉 : {} 원".format(int(salary_data["avg_salary"])))
    elif flag == 0:
        print("기업명 :  {}".format(search_by_name), end="\n")
        print("찾으시는 기업의 연봉 정보가 없습니다.")
        
    return salary_data

In [ ]:
0. '자소설닷컴'의 자료 형태 파악
  1) 자소설 채용 달력 (자료 형태: json)
    - 설정한 기간 중 모든 채용 공고 제공
    - param = 검색을 시작할 일자와 종료 일자의 'date'
  2) 각 기업의 세부 공고 (자료 형태: json)
    - 해당 공고에 대한 보다 자세한 내용(채용 형태 등)
    - param = 1)에서 얻은 공고의 'id'
  3) 각 기업의 지원자 정보 (자료 형태: json)
    - 선택한 기업에 자소서 작성을 완료한 지원자 정보 제공
    - param = 로그인 세션, 2)에의 세부 공고에서 얻는 'company_group_id'

In [ ]:
def find_in_jasoseol ((int)search_month = 1, (str)*search_by_name):
        
   return (list) result  < len(result)=3  
            [0] = 자소설 채용 달력
            [1] = 디테일한 공고
            [2] = 해당 기업에 지원한 지원자 정보

input = 공고를 찾고자 하는 기간 (default = 1, 최대 12개월까지 가능)
output = 1) 입력한 기간 중 자소설 닷컴의 모든 채용 공고를 return
         2) 회사명(search_by_name)을 입력한 경우, 
            해당 기간 중 모든 채용 공고를
            회사의 디테일한 공고 정보 및 지원자 정보 return

In [ ]:
1. Input 값 핸들링
  1) search_month : 공고를 검색할 기간 값 설정
        기본값(입력이 없을 경우) 1이며, 최대 12까지 설정 가능
  2) search_by_name : 자세한 정보를 검색할 기업명 입력
        기본값(입력이 없을 경우) return 값은 없음
        검색에 실패한 경우 '해당 기업 공고가 없음'을 return

In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import json
import datetime
import copy
import pandas as pd
import numpy as np

In [ ]:
# 함수 입력 값 컨트롤하기 
search_year = datetime.date.today().year
search_day = datetime.date.today().day

# 검색 '월' error handling
if type(search_month) != int:
    print("잘못된 값입니다.")
#     return
elif (search_month > 12) :
    print("12개월 이내의 공고만 검색가능합니다.")
#     return
elif datetime.date.today().month - search_month < 1 :
    search_month = datetime.date.today().month - search_month + 12
    search_year-= 1 
else:
    search_month = datetime.date.today().month - search_month



In [ ]:
# 검색 조건 결정
start_date = datetime.date(search_year, search_month, search_day)
end_date = datetime.date.today()

# 기업명 결정하기
if search_by_name == ():
    search_by_name = ""
else: search_by_name = search_by_name[0]

In [ ]:
2-1. 채용 달력 정보 받아오기
    - 입력 받은 검색 시작/종료 일을 토대로 채용 달력 정보 가져오기
    - 검색할 기업명이 입력되지 않았을 경우, 채용 달력 정보만 return 후 종료
    - 검색할 기업명이 입력됐으나 공고가 없으면 채용 달력 정보만 return 후 종료

In [ ]:
# jasoseol 사이트에서 채용 달력 정보 받아오기
data = {
    'end_time':"{}T15:00:00.000Z".format(end_date),
    'start_time':"{}T15:00:00.000Z".format(start_date)
}
resp_jasoseol_calendar = requests.post("https://jasoseol.com/employment/calendar_list.json", data=data)
calendar_all = pd.DataFrame(resp_jasoseol_calendar.json()["employment"])

# 기업명이 입력되지 않았을 때 > 채용 달력 정보만 제공하고 return
if search_by_name == '':
    pass
#     return calendar_all


In [ ]:
# 기업명이 입력되었을 때 > 
#  1) Detailed 공고 정보 제공
#    가. 찾는 기업의 'id' 찾기
i=0
id_list = []
flag=-1

for i in range(len(resp_jasoseol_calendar.json()["employment"])):
    if search_by_name == resp_jasoseol_calendar.json()["employment"][i]["name"]:
        id_list.append(resp_jasoseol_calendar.json()["employment"][i]["id"])
        flag+=1

# error handling (공고가 없을 경우!)
if flag == -1:
    print("해당 기간 중 찾으시는 기업의 채용 공고가 없습니다.")
#         return calendar_all


2-2. 기업별 구체적인 공고 가져오기
    - 2-1.에서 얻은 'id'를 바탕으로 for문을 통해 기업정보 가져오기
    - 중복된 공고가 있을 수 있으므로, 1건을 찾더라도 종료하지 않고 계속 반복
    - 단, 중복된 공고인 경우에도 'company_group_id'는 같기 때문에 int로 변수 설정

In [ ]:
detail_each = []
group_id = 0
for i in range(len(id_list)):
    data = {"employment_company_id":id_list[i]}
    resp_jasoseol_detail = requests.post("https://jasoseol.com/employment/get.json", data=data)
    detail_each.append(resp_jasoseol_detail.json())
    group_id = resp_jasoseol_detail.json()['recruit_refs'][0]['company_group_id']

detail_all = pd.DataFrame(detail_each)

#    나. (jasoseol_detail) 기업 id를 통해 구체적 채용 정보 가져오기
detail_each = []
group_id = 0
for i in range(len(id_list)):
    data = {"employment_company_id":id_list[i]}
    resp_jasoseol_detail = requests.post("https://jasoseol.com/employment/get.json", data=data)
    detail_each.append(resp_jasoseol_detail.json())
    group_id = resp_jasoseol_detail.json()['recruit_refs'][0]['company_group_id']

detail_all = pd.DataFrame(detail_each)

2-3. 해당 기업 지원자 정보 가져오기
    - 2-2.에서 얻은 'company_group_id'를 통해 해당 기업 지원자 정보 가져오기
    - 기업 지원자 정보는 로그인한 상태에서만 조회가 가능하므로 로그인 세션 생성

In [ ]:
#  2) (data_lab) 기업 공고 지원자 정보 가져오기
#    가. log-in session 만들기
applicant_each = []
applicant_all = []
login_session = requests.Session()
data = {
    'email': 'gdpark92@gmail.com',
    'password':'qkfka123!@'        
}
resp_data_lab = login_session.post("https://jasoseol.com/sign/log_in.json", data=data)

In [ ]:
#    나. 기업 공고 지원자 정보 가져오기
for i in range(len(id_list)):
    headers = {
        'Origin': 'https://jasoseol.com',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Referer':"https://jasoseol.com/datalab/{}?ec={}".format(group_id, id_list[i]),
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }
    data = {"company_group_id":str(group_id)}
    resp_data_lab = login_session.post('https://jasoseol.com/datalab/get_data.json', data=data, headers=headers)
    applicant_all.append(resp_data_lab.json()["result"]["employment_companies"][0]["employments"])

print(resp_data_lab.json()["result"]["employment_companies"][0]["employments"])

In [ ]:
3. 최종 결과값 반환
    - 2-1. 2-2. 2-3. 에서 각각 구한 정보를 dataframe으로 list에 저장하여 반환

In [ ]:
# 최종 결과를 받는 List return
result=[]
result.append(calendar_all)
result.append(detail_all)
result.append(applicant_all)

#     return result

In [3]:
find_in_jasoseol()

,id,name,start_time,end_time,image_file_name,recruit_type,business_size,business_type,employments,in24hours
0,36061,틱톡(TikTok),2020-06-05T00:00:00.000+09:00,2020-07-31T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 80}], 'id': 150...",False
1,36205,링크제니시스,2020-04-04T12:00:00.000+09:00,2020-06-03T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 31}, {'group_id...",False
2,36410,패스트캠퍼스,2020-03-05T14:01:00.000+09:00,2020-05-31T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 13}], 'id': 151...",False
3,36449,패스트캠퍼스,2020-03-10T11:40:00.000+09:00,2020-05-31T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 16}, {'group_id...",False
4,36450,패스트캠퍼스,2020-03-10T11:50:00.000+09:00,2020-05-31T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 16}, {'group_id...",False
...,...,...,...,...,...,...,...,...,...,...
545,37645,빅히트엔터테인먼트,2020-06-12T11:08:00.000+09:00,2020-06-26T23:55:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 70}, {'group_id...",True
546,37646,KB증권,2020-06-12T11:23:00.000+09:00,2020-06-22T17:00:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 10}, {'group_id...",True
547,37647,현대그린푸드,2020-06-12T11:47:00.000+09:00,2020-06-22T17:00:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,big_business,None,"[{'duty_groups': [{'group_id': 76}, {'group_id...",True
548,37648,EY한영,2020-06-12T11:41:00.000+09:00,2020-06-15T23:59:00.000+09:00,https://jasoseol.s3-ap-northeast-2.amazonaws.c...,0,None,None,"[{'duty_groups': [{'group_id': 48}], 'id': 156...",True


In [4]:
find_in_jasoseol(1, "EY한영")

[        id        name                     start_time  \
 0    36061  틱톡(TikTok)  2020-06-05T00:00:00.000+09:00   
 1    36205      링크제니시스  2020-04-04T12:00:00.000+09:00   
 2    36410      패스트캠퍼스  2020-03-05T14:01:00.000+09:00   
 3    36449      패스트캠퍼스  2020-03-10T11:40:00.000+09:00   
 4    36450      패스트캠퍼스  2020-03-10T11:50:00.000+09:00   
 ..     ...         ...                            ...   
 545  37645   빅히트엔터테인먼트  2020-06-12T11:08:00.000+09:00   
 546  37646        KB증권  2020-06-12T11:23:00.000+09:00   
 547  37647      현대그린푸드  2020-06-12T11:47:00.000+09:00   
 548  37648        EY한영  2020-06-12T11:41:00.000+09:00   
 549  37649       NS홈쇼핑  2020-06-12T12:35:00.000+09:00   
 
                           end_time  \
 0    2020-07-31T23:59:00.000+09:00   
 1    2020-06-03T23:59:00.000+09:00   
 2    2020-05-31T23:59:00.000+09:00   
 3    2020-05-31T23:59:00.000+09:00   
 4    2020-05-31T23:59:00.000+09:00   
 ..                             ...   
 545  2020-06-26T23:55:00.000

In [5]:
find_in_jasoseol(13)

12개월 이내의 공고만 검색가능합니다.


In [6]:
find_in_jasoseol("없는기업")

잘못된 값입니다.


In [ ]:
0. KreditJob 의 자료 형태 파악
    - 기업별 신입/평균 연봉 정보를 json으로 제공
    - 1회 조회 시 20개 기업씩 제공 (pagingIndex)
    - 공기업(alio)과 사기업(fss)로 2개의 데이터 셋 확인

In [ ]:
1. Input 값 핸들링
    - (str)search_by_name /*필수/ 

In [ ]:
# def find_salary(search_by_name):    
search_by_name=search_by_name.strip()
if search_by_name =="":
    print("연봉 정보를 검색할 기업명을 입력하세요")
    return

# import
import requests
from bs4 import BeautifulSoup as BS
import json

# 결과값을 담을 변수 선언
result_li_alio = []
result_li_fss = []

salary_data ={}
j=0
k=0
flag=0

In [ ]:
2. 연봉 정보 검색하기
    - 공기업(alio) / 사기업(fss)의 2개 정보
    - 검색 속도 향상을 위해 공기업 > 사기업으로 검색하며 결과값 확인 즉시 종료

In [ ]:
while(1):
    url_alio = "https://kreditjob.com/api/company/alioList?alioType=&order=ASC&pagingIndex={}&pagingNum=20&sortAlio=CMPN_NM".format(j)
    resp_alio = requests.get(url_alio)

    # 자료 값을 다 찾았을 때 loop 탈출
    if len(resp_alio.json()["data"]) == 0:
        break

    # 20개 단위로 resul_li 에 추가하기
    result_li_alio.append((resp_alio.json()["data"]))   

    # 기업명으로 검색하기
    for i in range(len(result_li_alio[k])):        
        if result_li_alio[k][i]["CMPN_NM"] == search_by_name:
            salary_data = {
                "id":result_li_alio[k][i]["CMPN_NM"], 
                "rookey_salary":result_li_alio[k][i]["ROOKEY_SALARY_YY"],  
                "avg_salary":result_li_alio[k][i]["AVG_SALARY_YY"]
            }
            flag = 1
            break    

    # 다음 반복을 위한 변수값 재설정        
    j+=20
    k+=1

    # 검색에 성공했을 때 loop 탈출
    if flag == 1:
        break


In [ ]:
# 사기업 검색을 위한 변수 _init_
j=0
k=0
flag=0     

# 공기업에서 찾은 경우 검색 생략
if flag == 0:
    while(1):
        url_fss = "https://kreditjob.com/api/company/fssList?fssType=&order=ASC&pagingIndex={}&pagingNum=20&sortFss=CMPN_NM".format(j)
        resp_fss = requests.get(url_fss)

        # 자료 값을 다 찾았을 때 loop 탈출
        if len(resp_fss.json()["data"]) == 0:
            break

        # 20개 단위로 resul_li 에 추가하기
        result_li_fss.append((resp_fss.json()["data"]))   

        # 기업명으로 검색하기
        for i in range(len(result_li_fss[k])):        
            if result_li_fss[k][i]["CMPN_NM"] == search_by_name:
                salary_data = {
                    "id":result_li_fss[k][i]["CMPN_NM"], 
                    "rookey_salary":result_li_fss[k][i]["ROOKEY_SALARY_FSS"],  
                    "avg_salary":result_li_fss[k][i]["AVG_SALARY_FSS"]
                }
                flag = 1
                break    

        # 다음 반복을 위한 변수값 재설정        
        j+=20
        k+=1

        # 검색에 성공했을 때 loop 탈출
        if flag == 1:
            break



In [ ]:
3. 결과값 반환

In [ ]:
# 검색에 성공했을 때 값 return
if flag == 1:
    print("기업명 :  {}".format(search_by_name), end="\n")
    print(" - 신입 연봉 : {} 원".format(int(salary_data["rookey_salary"])))
    print(" - 평균 연봉 : {} 원".format(int(salary_data["avg_salary"])))
elif flag == 0:
    print("기업명 :  {}".format(search_by_name), end="\n")
    print("찾으시는 기업의 연봉 정보가 없습니다.")

return salary_data

In [7]:
find_salary("없는기업")

기업명 :  없는기업
찾으시는 기업의 연봉 정보가 없습니다.


{}

In [8]:
find_salary()

TypeError: find_salary() missing 1 required positional argument: 'search_by_name'

In [ ]:
find_salary("삼성전자")